# Multiple Dataset Sentiment Analysis

In [43]:
import pandas as pd 
imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)

In [44]:
imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [45]:
colums = ['text', 'label']
imdb.columns = colums

In [46]:
imdb.head()

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [47]:
yelp = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
columns = ['text', 'label']
yelp.columns = columns

In [48]:
amazon = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
columns = ['text', 'label']
amazon.columns = columns

In [49]:
data = yelp.append([amazon, imdb], ignore_index=True)

In [50]:
data.head()

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [51]:
data.shape

(2496, 2)

In [52]:
df = data.sample(frac=1).reset_index(drop=True)

In [53]:
df.head()

,text,label
0,Highly unrecommended.,0
1,"I love Lane, but I've never seen her in a movi...",0
2,"I saw ""Mirrormask"" last night and it was an un...",0
3,you could only take 2 videos at a time and the...,0
4,but the movie makes a lot of serious mistakes.,0


In [54]:
import string 
star = string.punctuation
star

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [55]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [56]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords= list(STOP_WORDS)
stopwords

['whole',
 'our',
 'hers',
 'on',
 "n't",
 'others',
 'his',
 'nothing',
 'upon',
 'even',
 'anyhow',
 'when',
 'who',
 'they',
 'may',
 'two',
 'if',
 'for',
 'thus',
 'neither',
 'several',
 'me',
 'the',
 'least',
 'which',
 'toward',
 'seemed',
 'did',
 'twenty',
 'anyway',
 'four',
 'us',
 'now',
 'under',
 'being',
 'along',
 'well',
 'other',
 'various',
 'became',
 'how',
 'last',
 'keep',
 'with',
 'using',
 'latterly',
 'over',
 'name',
 'make',
 'noone',
 'each',
 "'d",
 'might',
 'otherwise',
 'nevertheless',
 'a',
 'ever',
 'thereby',
 'thereafter',
 'therefore',
 'both',
 'seem',
 'across',
 'whenever',
 'my',
 'no',
 'them',
 'whereas',
 'because',
 'empty',
 'ca',
 'most',
 'everything',
 'put',
 'show',
 'few',
 'ours',
 'fifty',
 '‘d',
 'meanwhile',
 '‘m',
 'ourselves',
 'very',
 'by',
 'used',
 'their',
 'nor',
 'has',
 'had',
 'whither',
 'get',
 'she',
 'have',
 'from',
 'sixty',
 'down',
 'this',
 'go',
 'mostly',
 'whereby',
 'becoming',
 'done',
 'therein',
 'af

In [57]:
from spacy.lang.en import English
# load English tokenizer,  tagger, parser, NER, and work vector:
parser = English()
def spacy_tokenizer(sentance):
    mytokens = parser(sentance)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in star]
    return mytokens
    

In [64]:
from sklearn.base import TransformerMixin
class predictors(TransformerMixin):
    def transform(self, X, **tranform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self
    
    def get__params(self, Deep=True):
        return {}

def clean_text(text):
    return text.strip().lower()

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [60]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [61]:
X = data['text']
ylabels = data['label']

In [62]:
xtrain , xtext, ytrain , ytest = train_test_split(X,ylabels, test_size=0.3, random_state = 42)

In [65]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation|
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000002BFC2F9FAF0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000002BFC2F95820>)),
                ('classifier', LogisticRegression())])

In [69]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(xtext)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(ytest, predicted))
print("Logistic Regression Precision:",metrics.precision_score(ytest, predicted))
print("Logistic Regression Recall:",metrics.recall_score(ytest, predicted))

Logistic Regression Accuracy: 0.855807743658211
Logistic Regression Precision: 0.8736263736263736
Logistic Regression Recall: 0.8368421052631579


In [77]:
pipe.predict(['this is bad'])

array([0], dtype=int64)

In [71]:
pipe.predict(['Worth of watching it. Please like it'])

array([1], dtype=int64)

In [73]:
pipe.predict(['Loved it. amazing'])

array([1], dtype=int64)